# Subword-level UD

In [1]:
from IPython.display import HTML
from collections import namedtuple
from typing import List, Optional
from dataclasses import dataclass
from copy import deepcopy

from dep_tregex.ya_dep import visualize_tree

## Defining classes for storing tokens and trees

In [2]:
@dataclass
class CONLLUToken:
    idx: str
    form: str
    lemma: str = "_"
    upos: str = "_"
    xpos: str = "_"
    feats: str = "_"
    head: str = "0"
    deprel: str = "root"
    deps: str = "_"
    misc: str = "_"

    def __str__(self):
        return "\t".join([
            self.idx, self.form, self.lemma, self.upos, self.xpos,
            self.feats, self.head, self.deprel, self.deps, self.misc
        ])

    def __repr__(self):
        return self.__str__()

    @classmethod
    def from_str(cls, line: str):
        return cls(*line.split("\t"))

In [3]:
class CONLLUTree:
    def __init__(
            self,
            tokens: List[CONLLUToken],
            sent_id: str = "",
            sent_text: Optional[str] = None,
            root_idx: Optional[int] = None
    ):
        self.tokens = tokens

        if root_idx is not None:
            self.root_idx = int(root_idx)
        else:
            for i, token in enumerate(tokens):
                if token.head == "0":
                    self.root_idx = i

        self.sent_id = sent_id
        self.sent_text = sent_text or ' '.join([token.form for token in tokens])

    def __str__(self):
        return "\n".join(
            [
                f'# sent_id = {self.sent_id}',
                f'# text = {self.sent_text}'
            ] + [str(token) for token in self.tokens]
        )

    def __repr__(self):
        return self.__str__()

    def html(self, fpath: Optional[str] = None):
        content = visualize_tree("\n".join([str(token) for token in self.tokens]))
        if fpath:
            with open(fpath, "w") as f:
                f.write(content)
        return content

    @classmethod
    def from_text(cls, text: str):
        lines = text.strip().split("\n")
        tokens = [
            CONLLUToken.from_str(line)
            for line in lines if not line.startswith("#")
        ]
        return cls(tokens)

    def __len__(self):
        return len(self.tokens)

    def __iter__(self):
        iter(self.tokens)

    def __getitem__(self, item):
        return self.tokens[item]

In [4]:
def merge_trees(
        tree_l: CONLLUTree, tree_r: CONLLUTree,
        deprel: str, is_arc_l2r: bool = True
):
    tokens_l = deepcopy(tree_l.tokens)
    tokens_r = deepcopy(tree_r.tokens)
    l = len(tree_l.tokens)
    for t in tokens_r:
        t.idx = str(int(t.idx) + l)
        if int(t.head) > 0:
            t.head = str(int(t.head) + l)
    if is_arc_l2r:
        tokens_r[tree_r.root_idx].head = tokens_l[tree_l.root_idx].idx
        tokens_r[tree_r.root_idx].deprel = deprel
        root_idx = tree_l.root_idx
    else:
        tokens_l[tree_l.root_idx].head = str(
            int(tree_r.tokens[tree_r.root_idx].idx) + l
        )
        tokens_l[tree_l.root_idx].deprel = deprel
        root_idx = tree_r.root_idx + l
    return CONLLUTree(tokens_l + tokens_r, root_idx=root_idx)


## Defining classes for storing information about word formation

In [5]:
WFToken = namedtuple(
    "WFToken", 
    ["derived_from_lemma", "derived_from_upos", "rule_id"]
)
WFCompoundToken = namedtuple(
    "WFCompoundToken", 
    ["derived_from_lemma", "derived_from_upos", "derived_from_modifiers", "rule_id"]
)

RuleInfo = namedtuple(
    "RuleInfo", 
    ["short_id", "info", "pos_b", "pos_a"]
)
ComplexRuleInfo = namedtuple(
    "ComplexRuleInfo", 
    ["short_id", "info", "pos_b", "pos_a", "simple_rules"]
)
CompoundRuleInfo = namedtuple(
    "CompoundRuleInfo", 
    ["short_id", "info", "pos_b", "pos_a", "head_rules", "modifier_rules", "after_rules"]
)


In [6]:
# TODO: create this dict automatically
rules_by_ids = {
    "при-": RuleInfo("при-", "PFX", "VERB", "VERB"),
    "на-": RuleInfo("на-", "PFX", "VERB", "VERB"),
    "об-": RuleInfo("об-", "PFX", "VERB", "VERB"),
    "-0N": RuleInfo("-0N", "SFX", "VERB", "NOUN"),
    "-н(ый)": RuleInfo("-н(ый)", "SFX", "NOUN", "ADJ"),
    "-ов(ый)": RuleInfo("-ов(ый)", "SFX", "NOUN", "ADJ"),
    "rule977(adj + о -> adv)": RuleInfo("-о", "SFX", "ADJ", "ADV"),
    "-ость": RuleInfo("-ость", "SFX", "ADJ", "NOUN"),
    "N": RuleInfo("N", "CONV", "ADJ", "NOUN"),
    "rule994(по + adj + ому -> adv)": ComplexRuleInfo(
        "по--ому", "PFX,SFX", "ADJ", "ADV", [
            RuleInfo("-ому", "SFX", "ADJ", "ADV"),
            RuleInfo("по-", "PFX", "ADV", "ADV"),
        ]
    ),
    "A+A": CompoundRuleInfo(
        "A+A", "COMPOUND", "ADJ", "ADJ", [], [], []
    ),
    "N+A": CompoundRuleInfo(
        "N+A", "COMPOUND", "ADJ", "ADJ", [], [], []
    ),
    "U+N+н(ый)": CompoundRuleInfo(
        "U+N+н(ый)", "COMPOUND,SFX", "NOUN", "ADJ", [RuleInfo("-н(ый)", "SFX", "NOUN", "ADJ")], [], []
    ),
    "A+N+н(ый)": CompoundRuleInfo(
        "A+N+н(ый)", "COMPOUND,SFX", "NOUN", "ADJ", [RuleInfo("-н(ый)", "SFX", "NOUN", "ADJ")], [], []
    )
}


derivbase = {
    "принять": WFToken("*ять", "VERB", "при-"),
    "прием": WFToken("принять", "VERB", "-0N"),
    "свободный": WFToken("свобода", "NOUN", "-н(ый)"),
    "приемный": WFToken("прием", "NOUN", "-н(ый)"),
    "приемная": WFToken("приемный", "ADJ", "N"),
    "обставить": WFToken("ставить", "VERB", "об-"),
    "просто": WFToken("простой", "ADJ", "rule977(adj + о -> adv)"),
    "дело": WFToken("делать", "VERB", "-0N"),
    "деловой": WFToken("дело", "NOUN", "-ов(ый)"),
    "по-деловому": WFToken("деловой", "ADJ", "rule994(по + adj + ому -> adv)"),
    "достоверность": WFToken("достоверный", "ADJ", "-ость"),
    "благородно": WFToken("благородный", "ADJ", "rule977(adj + о -> adv)"),
    "самозабвенно": WFToken("самозабвенный", "ADJ", "rule977(adj + о -> adv)"),
    "еда": WFToken("есть", "VERB", "-0N"),
    "сон": WFToken("спать", "VERB", "-0N"),
    "наука": WFToken("научить", "VERB", "-0N"),
    "научить": WFToken("учить", "VERB", "на-"),

    
    "достоверный": WFCompoundToken("верный", "ADJ", ("достойный",), "A+A"),
    "самозабвенный": WFCompoundToken("забвенный", "ADJ", ("себя",), "N+A"),
    "многочисленный": WFCompoundToken("число", "NOUN", ("много",), "U+N+н(ый)"),
    "белоколонный": WFCompoundToken("колона", "NOUN", ("белый",), "A+N+н(ый)"),
    "благородный": WFCompoundToken("род", "NOUN", ("благой",), "A+N+н(ый)"),
}

In [7]:
def merge_with_simple_rule(stem_tree: CONLLUTree, rule: RuleInfo) -> CONLLUTree:
    affix_tree = CONLLUTree([CONLLUToken("1", rule.short_id, rule.short_id)])
    if rule.info == "SFX":
        if rule.pos_a != rule.pos_b:
            tree = merge_trees(stem_tree, affix_tree, deprel="sfx", is_arc_l2r=False)
        else:
            # TODO: separate modifiers from other suffixes
            tree = merge_trees(stem_tree, affix_tree, deprel="msfx", is_arc_l2r=True)
    elif rule.info == "PTFX":
        tree = merge_trees(stem_tree, affix_tree, deprel="refl", is_arc_l2r=True)
    elif rule.info == "PFX":
        tree = merge_trees(affix_tree, stem_tree, deprel="mpfx", is_arc_l2r=False)
    elif rule.info == "CONV":
        tree = merge_trees(stem_tree, affix_tree, deprel="conv", is_arc_l2r=False)
    else:
        raise AssertionError('Rule info is incorrect!', rule.short_id, rule.info)
    return tree


def merge_with_complex_rule(stem_tree: CONLLUTree, rule: ComplexRuleInfo) -> CONLLUTree:
    tree = stem_tree
    for simple_rule in rule.simple_rules:
        tree = merge_with_simple_rule(tree, simple_rule)
    return tree


def make_wf_tree(word: str) -> CONLLUTree:
    if word not in derivbase:
        return CONLLUTree([CONLLUToken("1", word, word)])
    wf_token = derivbase[word]
    stem_tree = make_wf_tree(wf_token.derived_from_lemma)
    rule = rules_by_ids[wf_token.rule_id]
    if isinstance(rule, RuleInfo):
        return merge_with_simple_rule(stem_tree, rule)
    elif isinstance(rule, ComplexRuleInfo):
        return merge_with_complex_rule(stem_tree, rule)
    elif isinstance(rule, CompoundRuleInfo):
        modifiers_trees = [make_wf_tree(m) for m in wf_token.derived_from_modifiers]
        # TODO: rules for modifiers
        for hrule in rule.head_rules:
            stem_tree = merge_with_simple_rule(stem_tree, hrule)
        for modifier_tree in reversed(modifiers_trees):  # (m1 (m2 (m3 h))) sfx
            stem_tree = merge_trees(modifier_tree, stem_tree, "compound", is_arc_l2r=False)
        for hrule in rule.after_rules:  # prefix?
            stem_tree = merge_with_simple_rule(stem_tree, hrule)
        return stem_tree
    else:
        raise NotImplementedError


def make_wfdt(text: str):
    word_tree = CONLLUTree.from_text(text)
    subword_trees = []
    subword_roots = [-1]
    cur_len = 0
    united_subword_tokens = []
    for token in word_tree.tokens:
        subword_tree = make_wf_tree(token.lemma)
        subword_roots.append(cur_len + subword_tree.root_idx)
        subword_trees.append(subword_tree)
        for subword_token in subword_tree.tokens:
            subword_token.idx = str(len(united_subword_tokens) + 1)
            subword_token.head = str(int(subword_token.head) + cur_len)
            united_subword_tokens.append(subword_token)
        cur_len += len(subword_tree)

    renumerated = {"0": 0}
    for i, token in enumerate(word_tree.tokens):
        renumerated[token.idx] = i + 1

    for i, token in enumerate(word_tree.tokens):
        renum_head = renumerated[token.deps.split(":")[0]]
        # if token.idx.isdigit():
        #     renum_head = renumerated[token.head]
        # else:
        #     # 33.1	_	_	_	_	_	_	_	3:conj	_
        #     renum_head = renumerated[token.deps.split(":")[0]]
        united_subword_tokens[subword_roots[i+1]].head = str(subword_roots[renum_head]+1)
        united_subword_tokens[subword_roots[i+1]].deprel = token.deprel

    return CONLLUTree(united_subword_tokens)


## Now let's have a look at some examples!

In [8]:
text = """
1	Приемная	приемная	NOUN	_	Animacy=Inan|Case=Nom|Gender=Fem|Number=Sing	3	nsubj:pass	3:nsubj:pass	_
2	была	быть	AUX	_	Aspect=Imp|Gender=Fem|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Act	3	aux:pass	3:aux:pass	_
3	обставлена	обставить	VERB	_	Aspect=Perf|Gender=Fem|Number=Sing|Tense=Past|Variant=Short|VerbForm=Part|Voice=Pass	0	root	0:root	_
4	просто	просто	ADV	_	Degree=Pos	3	advmod	3:advmod	SpaceAfter=No
5	,	,	PUNCT	_	_	7	punct	7:punct	_
6	но	но	CCONJ	_	_	7	cc	7:cc	_
7	по-деловому	по-деловому	ADV	_	Degree=Pos	4	conj	4:conj	SpaceAfter=No
8	.	.	PUNCT	_	_	3	punct	3:punct	_

""".strip()

In [9]:
t = CONLLUTree.from_text(text)

In [10]:
HTML(t.html())

In [11]:
HTML(make_wf_tree("многочисленный").html("многочисленный.html"))

In [12]:
HTML(make_wf_tree("достоверный").html())

In [13]:
HTML(make_wf_tree("по-деловому").html())

In [14]:
HTML(make_wf_tree("еда").html())

In [15]:
HTML(
    merge_trees(
        make_wf_tree("белоколонный"), 
        make_wf_tree("здание"),
        deprel="amod", is_arc_l2r=False
    ).html()
)

In [16]:
HTML(
    merge_trees(
        make_wf_tree("обставить"), 
        make_wf_tree("по-деловому"),
        deprel="advmod", is_arc_l2r=True
    ).html()
)

In [17]:
HTML(make_wfdt(text).html("priem.html"))

In [18]:
HTML(make_wfdt("""
# sent_id = uppsalaBitov_1.xml_196
# text = Но почти ничего с материальной достоверностью зрения.
1	Но	но	CCONJ	_	_	3	cc	3:cc	_
2	почти	почти	ADV	_	Degree=Pos	3	obl	3:obl	_
3	ничего	ничто	PRON	_	Case=Gen	0	root	0:root	_
4	с	с	ADP	_	_	6	case	6:case	_
5	материальной	материальный	ADJ	_	Case=Ins|Degree=Pos|Gender=Fem|Number=Sing	6	amod	6:amod	_
6	достоверностью	достоверность	NOUN	_	Animacy=Inan|Case=Ins|Gender=Fem|Number=Sing	3	nmod	3:nmod	_
7	зрения	зрение	NOUN	_	Animacy=Inan|Case=Gen|Gender=Neut|Number=Sing	6	nmod	6:nmod	SpaceAfter=No
8	.	.	PUNCT	_	_	3	punct	3:punct	_
""").html())

In [19]:
HTML(make_wfdt("""
# text = В них находил он необыкновенно приятный тон благопристойности и почтения и тогда воображал себя не иначе как уже стариком, окруженным многочисленными учениками, членом многочисленных ученых обществ, а собственную жизнь - каким-то непрерывным чествованием.
1	В	в	ADP	_	_	2	case	2:case	_
2	них	они	PRON	_	Case=Loc|Number=Plur|Person=3	3	obl	3:obl	_
3	находил	находить	VERB	_	Aspect=Imp|Gender=Masc|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Act	0	root	0:root	_
4	он	он	PRON	_	Case=Nom|Gender=Masc|Number=Sing|Person=3	3	nsubj	3:nsubj	_
5	необыкновенно	необыкновенно	ADV	_	Degree=Pos	6	obl	6:obl	_
6	приятный	приятный	ADJ	_	Animacy=Inan|Case=Acc|Degree=Pos|Gender=Masc|Number=Sing	7	amod	7:amod	_
7	тон	тон	NOUN	_	Animacy=Inan|Case=Acc|Gender=Masc|Number=Sing	3	obj	3:obj	_
8	благопристойности	благопристойность	NOUN	_	Animacy=Inan|Case=Gen|Gender=Fem|Number=Sing	7	nmod	7:nmod	_
9	и	и	CCONJ	_	_	10	cc	10:cc	_
10	почтения	почтение	NOUN	_	Animacy=Inan|Case=Gen|Gender=Neut|Number=Sing	8	conj	8:conj	_
11	и	и	CCONJ	_	_	13	cc	13:cc	_
12	тогда	тогда	ADV	_	Degree=Pos	13	advmod	13:advmod	_
13	воображал	воображать	VERB	_	Aspect=Imp|Gender=Masc|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Act	3	conj	3:conj	_
14	себя	себя	PRON	_	Case=Acc	13	obj	13:obj	_
15	не	не	PART	_	_	16	advmod	16:advmod	_
16	иначе	иначе	ADV	_	Degree=Pos	13	advmod	13:advmod	_
17	как	как	SCONJ	_	_	19	mark	19:mark	_
18	уже	уже	ADV	_	Degree=Pos	19	obl	19:obl	_
19	стариком	старик	NOUN	_	Animacy=Anim|Case=Ins|Gender=Masc|Number=Sing	16	obl	16:obl	SpaceAfter=No
20	,	,	PUNCT	_	_	21	punct	21:punct	_
21	окруженным	окружить	VERB	_	Aspect=Perf|Case=Ins|Gender=Masc|Number=Sing|Tense=Past|VerbForm=Part|Voice=Pass	19	acl	19:acl	_
22	многочисленными	многочисленный	ADJ	_	Case=Ins|Degree=Pos|Number=Plur	23	amod	23:amod	_
23	учениками	ученик	NOUN	_	Animacy=Anim|Case=Ins|Gender=Masc|Number=Plur	21	obl	21:obl	SpaceAfter=No
24	,	,	PUNCT	_	_	25	punct	25:punct	_
25	членом	член	NOUN	_	Animacy=Anim|Case=Ins|Gender=Masc|Number=Sing	19	conj	19:conj	_
26	многочисленных	многочисленный	ADJ	_	Case=Gen|Degree=Pos|Number=Plur	28	amod	28:amod	_
27	ученых	ученый	ADJ	_	Case=Gen|Degree=Pos|Number=Plur	28	amod	28:amod	_
28	обществ	общество	NOUN	_	Animacy=Inan|Case=Gen|Gender=Neut|Number=Plur	25	nmod	25:nmod	SpaceAfter=No
29	,	,	PUNCT	_	_	32	punct	32:punct	_
30	а	а	CCONJ	_	_	32	cc	33.1:cc	_
31	собственную	собственный	ADJ	_	Case=Acc|Degree=Pos|Gender=Fem|Number=Sing	32	amod	32:amod	_
32	жизнь	жизнь	NOUN	_	Animacy=Inan|Case=Acc|Gender=Fem|Number=Sing	3	conj	33.1:obj	_
33	-	-	PUNCT	_	_	36	punct	36:punct	_
33.1	_	_	_	_	_	_	_	3:conj	_
34	каким-то	какой-то	DET	_	Case=Ins|Gender=Neut|Number=Sing	36	det	36:det	_
35	непрерывным	непрерывный	ADJ	_	Case=Ins|Degree=Pos|Gender=Neut|Number=Sing	36	amod	36:amod	_
36	чествованием	чествование	NOUN	_	Animacy=Inan|Case=Ins|Gender=Neut|Number=Sing	32	orphan	33.1:obl	SpaceAfter=No
37	.	.	PUNCT	_	_	3	punct	3:punct	_
""").html("uppsalaBitov_2.xml_38.html"))

In [20]:
make_wfdt("""# sent_id = uppsalaBitov_2.xml_43
# text = Он представлял, как самозабвенно и благородно трудятся люди в этом большом белоколонном здании, а также в старинных, чуть ли не елизаветинских, деревянных домиках - лабораториях, разбросанных там и сям по прекрасному парку.
1	Он	он	PRON	_	Case=Nom|Gender=Masc|Number=Sing|Person=3	2	nsubj	2:nsubj	_
2	представлял	представлять	VERB	_	Aspect=Imp|Gender=Masc|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Act	0	root	0:root	SpaceAfter=No
3	,	,	PUNCT	_	_	8	punct	8:punct	_
4	как	как	SCONJ	_	_	8	mark	8:mark	_
5	самозабвенно	самозабвенно	ADV	_	Degree=Pos	8	advmod	8:advmod	_
6	и	и	CCONJ	_	_	7	cc	7:cc	_
7	благородно	благородно	ADV	_	Degree=Pos	5	conj	5:conj	_
8	трудятся	трудиться	VERB	_	Aspect=Imp|Mood=Ind|Number=Plur|Person=3|Tense=Pres|VerbForm=Fin|Voice=Mid	2	ccomp	2:ccomp	_
9	люди	человек	NOUN	_	Animacy=Anim|Case=Nom|Gender=Masc|Number=Plur	8	nsubj	8:nsubj	_
10	в	в	ADP	_	_	14	case	14:case	_
11	этом	этот	DET	_	Case=Loc|Gender=Neut|Number=Sing	14	det	14:det	_
12	большом	большой	ADJ	_	Case=Loc|Degree=Pos|Gender=Neut|Number=Sing	14	amod	14:amod	_
13	белоколонном	белоколонный	ADJ	_	Case=Loc|Degree=Pos|Gender=Neut|Number=Sing	14	amod	14:amod	_
14	здании	здание	NOUN	_	Animacy=Inan|Case=Loc|Gender=Neut|Number=Sing	8	obl	8:obl	SpaceAfter=No
15	,	,	PUNCT	_	_	27	punct	27:punct	_
16	а	а	CCONJ	_	_	27	cc	27:cc	_
17	также	также	PART	_	_	16	fixed	16:fixed	_
18	в	в	ADP	_	_	27	case	27:case	_
19	старинных	старинный	ADJ	_	Case=Loc|Degree=Pos|Number=Plur	27	amod	27:amod	SpaceAfter=No
20	,	,	PUNCT	_	_	24	punct	24:punct	_
21	чуть	чуть	ADV	_	Degree=Pos	23	advmod	23:advmod	_
22	ли	ли	PART	_	_	21	advmod	21:advmod	_
23	не	не	PART	_	_	24	advmod	24:advmod	_
24	елизаветинских	елизаветинский	ADJ	_	Case=Loc|Degree=Pos|Number=Plur	19	conj	19:conj	SpaceAfter=No
25	,	,	PUNCT	_	_	26	punct	26:punct	_
26	деревянных	деревянный	ADJ	_	Case=Loc|Degree=Pos|Number=Plur	19	conj	19:conj	_
27	домиках	домик	NOUN	_	Animacy=Inan|Case=Loc|Gender=Masc|Number=Plur	14	conj	14:conj	_
28	-	-	PUNCT	_	_	29	punct	29:punct	_
29	лабораториях	лаборатория	NOUN	_	Animacy=Inan|Case=Loc|Gender=Fem|Number=Plur	27	appos	27:appos	SpaceAfter=No
30	,	,	PUNCT	_	_	31	punct	31:punct	_
31	разбросанных	разбросить	VERB	_	Aspect=Perf|Case=Loc|Number=Plur|Tense=Past|VerbForm=Part|Voice=Pass	27	acl	27:acl	_
32	там	там	ADV	_	Degree=Pos	31	advmod	31:advmod	_
33	и	и	CCONJ	_	_	34	cc	34:cc	_
34	сям	сям	ADV	_	Degree=Pos	32	conj	32:conj	_
35	по	по	ADP	_	_	37	case	37:case	_
36	прекрасному	прекрасный	ADJ	_	Case=Dat|Degree=Pos|Gender=Masc|Number=Sing	37	amod	37:amod	_
37	парку	парк	NOUN	_	Animacy=Inan|Case=Dat|Gender=Masc|Number=Sing	31	obl	31:obl	SpaceAfter=No
38	.	.	PUNCT	_	_	2	punct	2:punct	_
""").html("uppsalaBitov_2.xml_43.html");

In [21]:
make_wfdt("""# sent_id = 2007Grechko.xml_253
# text = Для занятий наукой я выбирал "свободное" время, то есть выкраивал его за счет еды и сна.
1	Для	для	ADP	_	_	2	case	2:case	_
2	занятий	занятие	NOUN	_	Animacy=Inan|Case=Gen|Gender=Neut|Number=Plur	5	obl	5:obl	_
3	наукой	наука	NOUN	_	Animacy=Inan|Case=Ins|Gender=Fem|Number=Sing	2	nmod	2:nmod	_
4	я	я	PRON	_	Case=Nom|Number=Sing|Person=1	5	nsubj	5:nsubj	_
5	выбирал	выбирать	VERB	_	Aspect=Imp|Gender=Masc|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Act	0	root	0:root	_
6	"	"	PUNCT	_	_	7	punct	7:punct	SpaceAfter=No
7	свободное	свободный	ADJ	_	Case=Acc|Degree=Pos|Gender=Neut|Number=Sing	9	amod	9:amod	SpaceAfter=No
8	"	"	PUNCT	_	_	7	punct	7:punct	_
9	время	время	NOUN	_	Animacy=Inan|Case=Acc|Gender=Neut|Number=Sing	5	obj	5:obj	SpaceAfter=No
10	,	,	PUNCT	_	_	13	punct	13:punct	_
11	то	то	PRON	_	Animacy=Inan|Case=Nom|Gender=Neut|Number=Sing	13	mark	13:mark	_
12	есть	быть	VERB	_	Aspect=Imp|Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin|Voice=Act	11	fixed	11:fixed	_
13	выкраивал	выкраивать	VERB	_	Aspect=Imp|Gender=Masc|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Act	5	parataxis	5:parataxis	_
14	его	он	PRON	_	Case=Acc|Gender=Masc|Number=Sing|Person=3	13	obj	13:obj	_
15	за	за	ADP	_	_	17	case	17:case	_
16	счет	счет	NOUN	_	Animacy=Inan|Case=Acc|Gender=Masc|Number=Sing	15	fixed	15:fixed	_
17	еды	еда	NOUN	_	Animacy=Inan|Case=Gen|Gender=Fem|Number=Sing	13	obl	13:obl	_
18	и	и	CCONJ	_	_	19	cc	19:cc	_
19	сна	сон	NOUN	_	Animacy=Inan|Case=Gen|Gender=Masc|Number=Sing	17	conj	17:conj	SpaceAfter=No
20	.	.	PUNCT	_	_	5	punct	5:punct	_""").html("2007Grechko.xml_253.html");